# Data handling for football analytics

Goal of the notebook is to handle all necessery data for the project. Should download data, should sort data and create data sets for analytics.

## Imports below

In [1]:
# Add the imports needed
import requests
from bs4 import BeautifulSoup
import re
import urllib.request
import os
import numpy as np
import csv
import pandas as pd
import sys

# Constants needed

In [2]:
save_directory = r'C:\Users\theo_\Documents\Fotboll\FootballAnalyticsML\GameData'
countries = ['England', 'Spain', 'Italy', 'Germany', 'France', 'Turkey', 'Greece', 'Neterlands', 'Belgium']
url_start = "http://www.football-data.co.uk/"

# Help methods data downloading and storage

In [3]:
def get_country_url(country):
    return 'http://www.football-data.co.uk/%sm.php' % country.lower()

def get_country_path(country):
    return os.path.join(save_directory, country)

def check_create_path(path):
    if not os.path.exists(path):
        os.makedirs(path)
def get_all_files(directory, file_ending):
    return_files = list()
    for path, subdirs, files in os.walk(directory):
        for name in files:
            if file_ending in name:
                return_files.append(os.path.join(path, name))
    return return_files

# Data downloading
Data downloading from www.football-data.uk for all leagues in countries. 

## Current countries 

* England
* Spain
* Italy
* Germany
* France
* Turkey
* Greece
* Netherlands
* Belgium

In [9]:
def download_game_data():
    print('Begininning to download data for %d countries' % len(countries))
    print('---------------------------------------------------')
    for country in countries:
        print('Currently downloading data for country: %s' % country)
        download_country_data(country)
    print('---------------------------------------------------')
    print('Done with downloading data')

def download_country_data(country):
    rep = {"mmz4281/": "", "/": "_"}
    rep = dict((re.escape(k), v) for k, v in rep.items())
    pattern = re.compile("|".join(rep.keys()))
    
    country_path = get_country_path(country)
    check_create_path(country_path)
    country_url = get_country_url(country)
    files_downloaded = 0
    for a in scrape_links(country_url):
        if ".csv" in a['href']:
            name_of_file = pattern.sub(lambda m: rep[re.escape(m.group(0))], a['href'])
            league_path = os.path.join(country_path, a.string)
            check_create_path(league_path)
            urllib.request.urlretrieve(url_start + a['href'], os.path.join(league_path, name_of_file))
            files_downloaded += 1
    print('Done! Downloaded %d files' % files_downloaded)

def scrape_links(url):
    source_code = requests.get(url)
    plain_text = source_code.content
    soup = BeautifulSoup(plain_text, "lxml")
    return soup.find_all('a', href=True)


In [11]:
download_game_data()

Begininning to download data for 9 countries
---------------------------------------------------
Currently downloading data for country: England
Done! Downloaded 118 files
Currently downloading data for country: Spain
Done! Downloaded 49 files
Currently downloading data for country: Italy
Done! Downloaded 48 files
Currently downloading data for country: Germany
Done! Downloaded 52 files
Currently downloading data for country: France
Done! Downloaded 49 files
Currently downloading data for country: Turkey
Done! Downloaded 25 files
Currently downloading data for country: Greece
Done! Downloaded 25 files
Currently downloading data for country: Neterlands
Done! Downloaded 26 files
Currently downloading data for country: Belgium
Done! Downloaded 24 files
---------------------------------------------------
Done with downloading data


# Clean the data

The data will contain some not so nice things. We need to clean the data by for example removing empty rows

In [4]:
def remove_empty_rows(data_frame):
    data_frame = data_frame.dropna(how='all')
    return data_frame
def remove_empty_columns(data_frame):
    data_frame = data_frame.loc[:, ~data_frame.columns.str.contains('Unnamed')]
    return data_frame

def clean_dataframe(data_frame):
    data_frame = remove_empty_columns(data_frame)
    data_frame = remove_empty_rows(data_frame)
    return data_frame
 

df = pd.read_csv(r'C:\Users\theo_\Documents\Fotboll\FootballAnalyticsML\GameData\England\Division 1\0203_E1.csv', error_bad_lines=False)

In [ ]:
files = get_all_files(save_directory, '.csv')


class NullDevice():
    def write(self, s):
        pass

original_stderr = sys.stderr  # keep a reference to STDOUT

sys.stderr = NullDevice()  # redirect the real STDOUT
for file in files:
    try:
        df = pd.read_csv(file,  error_bad_lines=False)
        df = clean_dataframe(df)
        df.to_csv(file, index=False)

    except UnicodeDecodeError:
        print('Removing file: %s' % file)
        os.remove(file)

sys.stderr = original_stderr  # turn STDOUT back on

# Add important columns to each data set

We want to add some information to the data set. For example table location. We might also want to add ELO-ranking, goal difference etc. All methods for doing this should be coded below

## Help methods data handling

In [5]:
def get_teams_in_df(df):
    return pd.unique(df[['HomeTeam', 'AwayTeam']].values.ravel('K'))
def add_new_column(df, column_name, value=0):
    df[column_name] = value
    return df

def add_new_columns(df, columns, value=0):
    for column_name in columns:
        df = add_new_column(df, column_name, value)
    return df

# All data sets should have the same columns

In [43]:
files = get_all_files(save_directory, '.csv')
headers = list()
for file in files:
    df = pd.read_csv(file)
    a = df.columns
    headers = np.concatenate((headers,a))
    headers = np.unique(headers)

for file in files:
    df = pd.read_csv(file)
    current_head = df.columns
    for head in headers:
        if head not in current_head:
            df = add_new_column(df, head, np.nan)
    df.to_csv(file, index=False)

# Split data in to year, month, day
As an initial important task we have to split the date so that we write dates in one way. We need to split the date to year, month and day as to simplify the creation of one data set with all leagues where the data is ordered.


In [42]:
# Day, month, year
def get_date_type(date):
    date_split = split_date(date)
    if len(date_split[-1]) == 4:
        return 1
    elif len(date_split[-1]) == 2:
        return 2
    else:
        return 0
    
def split_date(date):
    return date.split('/')

def add_date(df,index, year, month, day):
    df.at[index, 'Year'] = year
    df.at[index, 'Month'] = month
    df.at[index, 'Day'] = day
    return df

def get_full_year(year):
    if int(year) > 70:
        return '19%s' % year
    else:
        return '20%s' % year

print('Adding separate date columns for %d files' %len(files))
for file in files:
    df = pd.read_csv(file,  error_bad_lines=False) 
    date_type = get_date_type(df.Date[0])
    
    if date_type == 0:
        print(first_date, file)
    else:
        df = add_new_columns(df, ['Year', 'Month', 'Day'])
        df = add_new_column(df, 'LastGame')
        for index, row in df.iterrows():
            if df.Date[index] == 'nan' or isinstance(df.Date[index],str)==False:
                df = add_date(df, index, df.Year[index-1],df.Month[index-1],df.Day[index-1])
                continue
            date_split = split_date(df.Date[index])
            if date_type == 2:
                date_split[2] = get_full_year(date_split[2])
            df = add_date(df, index, year=date_split[2], month=date_split[1], day=date_split[0])
        df.at[index, 'LastGame'] = 1
        df.to_csv(file, index=False)

print('------------------Done!-------------------')

Adding separate date columns for 409 files
------------------Done!-------------------


## Table location+Goal difference

In [63]:
class Table(object):
    
    
    def __init__(self, teams):
        # self.radius is an instance variable
        self.teams = teams

    def area(self):
        return self.pi * (self.radius ** 2)



def add_game_number(df):
    teams = get_teams_in_df(df)
    games_played = dict()

    for team in teams:
        games_played[team]=1
        
    df = add_new_column(df, 'HomeTeamGames')
    df = add_new_column(df, 'AwayTeamGames')
    for index, row in df.iterrows():
        HT = row['HomeTeam']
        AT = row['AwayTeam']
        df.at[index, 'HomeTeamGames'] = games_played[HT]
        df.at[index, 'AwayTeamGames'] = games_played[AT]
        games_played[HT] += 1
        games_played[AT] += 1
    return df


def add_table_position(df):
    pass
    

In [26]:
files = get_all_files(save_directory, '.csv')
for file in files:
    df = pd.read_csv(file)
